# Workflows de Análise de Dados e Geoestatística

## Módulo 2 - Análise Exploratória e Visualização de Dados com Python

O objetivo deste módulo e apresentar conceitos e funções iniciais da análise exploratória de dados (ou **EDA** - *exploratory data analysis*), utilizando a linguagem Python.

A idéia é apresentar o básico de bibliotecas voltadas a dados e utilizá-las para conhencermos os dados que estamos trabalhando.

### Imports

In [3]:
# paths
import os

# pandas
import pandas as pd

# data viz
import plotly
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

### Helper Functions